In [1]:
import pdfplumber

text = ""
with pdfplumber.open("SEJAL_CHOUREY(Resume_).pdf.pdf.pdf") as pdf:   
    for page in pdf.pages:
        text += page.extract_text() + "\n"

print(text)


FileNotFoundError: [Errno 2] No such file or directory: 'SEJAL_CHOUREY(Resume_).pdf.pdf.pdf'

In [10]:
!pip install -U spacy pdfplumber
!python -m spacy download en_core_web_sm

import os, re, json, random
from pathlib import Path
import pdfplumber
import pandas as pd
import spacy
from spacy.training.example import Example
from spacy.training import offsets_to_biluo_tags
from spacy.lang.en import English

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 1.2 MB/s eta 0:00:10
     -- ------------------------------------- 0.8/12.8 MB 1.2 MB/s eta 0:00:10
     --- ------------------------------------ 1.0/12.8 MB 1.1 MB/s eta 0:00:11
     ---- ----------------------------------- 1.3/12.8 MB 1.1 MB/s eta 0:00:11
     ---- ----------------------------------- 1.3/12.8 MB 1.1 MB/s eta 0:00:11
     ---- --------------------------------- 1.6/12.8 MB 912.1 kB/s eta 0:00:13
     ---- --------------------------------- 1.6/12.8 MB 912.1 kB/s eta 0:00:13
     ---- --------------------------------- 1.6/12.8 MB 912.1 kB/s eta 0:00:13
     ---- --------------------------------- 1.6/12.8 MB 912.1 kB/s eta 0:00:13
 

In [11]:
# only reads filename and text means all the data inside text 
pdf_folder = "pdf_files"  
rows = []
for fname in os.listdir(pdf_folder):
    if not fname.lower().endswith(".pdf"):
        continue
    path = os.path.join(pdf_folder, fname)
    text = ""
    try:
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print("Error reading", fname, e)
        continue
    rows.append({"filename": fname, "text": text})

df = pd.DataFrame(rows)
df.to_csv("resumes_extracted.csv", index=False)
print("Extracted", len(df), "resumes -> resumes_extracted.csv")


Extracted 118 resumes -> resumes_extracted.csv


In [12]:
df

,filename,text
0,ATS_DS_Resume_1.pdf,NAME: David Kim 91\nEmail: david_kim_91@mail.c...
1,ATS_DS_Resume_10.pdf,NAME: Manisha Yadav 45\nEmail: manisha_yadav_4...
2,ATS_DS_Resume_100.pdf,NAME: Ahmed Malik 47\nEmail: ahmed_malik_47@ma...
3,ATS_DS_Resume_11.pdf,NAME: Mia Johnson 46\nEmail: mia_johnson_46@ma...
4,ATS_DS_Resume_12.pdf,NAME: Mia Johnson 49\nEmail: mia_johnson_49@ma...
...,...,...
113,structured_resume_1.pdf,Aman Verma— SoftwareEngineer\n+91-9876543210|a...
114,structured_resume_2.pdf,NishthaGupta—DataAnalyst\n+91-9123456789|nisht...
115,structured_resume_3.pdf,RohitSingh—FrontendDeveloper\n+91-9987654321|r...
116,structured_resume_4 (1).pdf,PriyaMenon—MachineLearningEngineer\n+91-900001...


In [13]:
import os
import pdfplumber
import pandas as pd
import re

# ------------------ REGEX FUNCTIONS ------------------
def extract_email(text):
    match = re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-z]{2,}", text)
    return match.group(0) if match else ""

def extract_phone(text):
    match = re.search(r"\+?\d[\d\s\-]{8,15}", text)
    return match.group(0) if match else ""

def extract_name(text):
    lines = text.split("\n")
    first_line = lines[0].strip() if lines else ""

    # Remove common prefixes 
    first_line = re.sub(r'^(name[:\-]?\s*)', '', first_line, flags=re.IGNORECASE)

    # Remove numbers
    first_line = re.sub(r'\d+', '', first_line).strip()

    # Keep only alphabetic words
    words = re.findall(r'[A-Za-z]+', first_line)
    if 1 <= len(words) <= 4:
        return " ".join(words)

    return ""

SKILLS = ["python", "java", "sql", "excel", "flask", "django", "ml", "ai", "data analysis"]

def extract_skills(text):
    text_lower = text.lower()
    found = [skill for skill in SKILLS if skill in text_lower]
    return ", ".join(found)

def extract_cgpa(text):
    match = re.search(r"CGPA[:\s-]*([\d\.]+)", text, re.IGNORECASE)
    return match.group(1) if match else ""

# ------------------ PROCESS RESUME ------------------
def process_resume(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + '\n'
    return {
        "name": extract_name(text),
        "email": extract_email(text),
        "phone": extract_phone(text),
        "skills": extract_skills(text),
        "cgpa": extract_cgpa(text),
        "text": text
    }

# ------------------ MAIN ------------------
pdf_folder = "pdf_files"
all_rows = []

for fname in os.listdir(pdf_folder):
    if not fname.lower().endswith(".pdf"):
        continue
    path = os.path.join(pdf_folder, fname)
    try:
        row = process_resume(path)
        row["filename"] = fname  # add filename column
        all_rows.append(row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

# Convert to DataFrame
df = pd.DataFrame(all_rows)

# Save CSV with separate columns
df.to_csv("All_needed_entity.csv", index=False)
print("✔️ CSV saved: All_needed_entity.csv")
print("Total resumes processed:", df.shape[0])
print(df.head())


✔️ CSV saved: All_needed_entity.csv
Total resumes processed: 118
            name                      email             phone  \
0      David Kim      david_kim_91@mail.com  +91-6393136116\n   
1  Manisha Yadav  manisha_yadav_45@mail.com  +91-9610893319\n   
2    Ahmed Malik    ahmed_malik_47@mail.com  +91-7718575010\n   
3    Mia Johnson    mia_johnson_46@mail.com  +91-8120603783\n   
4    Mia Johnson    mia_johnson_49@mail.com  +91-7736163235\n   

                skills cgpa  \
0  python, sql, ml, ai        
1  python, sql, ml, ai        
2  python, sql, ml, ai        
3  python, sql, ml, ai        
4  python, sql, ml, ai        

                                                text               filename  
0  NAME: David Kim 91\nEmail: david_kim_91@mail.c...    ATS_DS_Resume_1.pdf  
1  NAME: Manisha Yadav 45\nEmail: manisha_yadav_4...   ATS_DS_Resume_10.pdf  
2  NAME: Ahmed Malik 47\nEmail: ahmed_malik_47@ma...  ATS_DS_Resume_100.pdf  
3  NAME: Mia Johnson 46\nEmail: mia_johnson_4

In [14]:
print("PDF Folder Path:", os.getcwd(), pdf_folder)
print("PDF Files:", os.listdir(pdf_folder))


PDF Folder Path: C:\Users\hp\Desktop\enhancement_NER_on100 pdf_files
PDF Files: ['ATS_DS_Resume_1.pdf', 'ATS_DS_Resume_10.pdf', 'ATS_DS_Resume_100.pdf', 'ATS_DS_Resume_11.pdf', 'ATS_DS_Resume_12.pdf', 'ATS_DS_Resume_13.pdf', 'ATS_DS_Resume_14.pdf', 'ATS_DS_Resume_15.pdf', 'ATS_DS_Resume_16.pdf', 'ATS_DS_Resume_17.pdf', 'ATS_DS_Resume_18.pdf', 'ATS_DS_Resume_19.pdf', 'ATS_DS_Resume_2.pdf', 'ATS_DS_Resume_20.pdf', 'ATS_DS_Resume_21.pdf', 'ATS_DS_Resume_22.pdf', 'ATS_DS_Resume_23.pdf', 'ATS_DS_Resume_24.pdf', 'ATS_DS_Resume_25.pdf', 'ATS_DS_Resume_26.pdf', 'ATS_DS_Resume_27.pdf', 'ATS_DS_Resume_28.pdf', 'ATS_DS_Resume_29.pdf', 'ATS_DS_Resume_3.pdf', 'ATS_DS_Resume_30.pdf', 'ATS_DS_Resume_31.pdf', 'ATS_DS_Resume_32.pdf', 'ATS_DS_Resume_33.pdf', 'ATS_DS_Resume_34.pdf', 'ATS_DS_Resume_35.pdf', 'ATS_DS_Resume_36.pdf', 'ATS_DS_Resume_37.pdf', 'ATS_DS_Resume_38.pdf', 'ATS_DS_Resume_39.pdf', 'ATS_DS_Resume_4.pdf', 'ATS_DS_Resume_40.pdf', 'ATS_DS_Resume_41.pdf', 'ATS_DS_Resume_42.pdf', 'ATS_DS_Re

In [15]:
print(df.head())
print(df.shape)


            name                      email             phone  \
0      David Kim      david_kim_91@mail.com  +91-6393136116\n   
1  Manisha Yadav  manisha_yadav_45@mail.com  +91-9610893319\n   
2    Ahmed Malik    ahmed_malik_47@mail.com  +91-7718575010\n   
3    Mia Johnson    mia_johnson_46@mail.com  +91-8120603783\n   
4    Mia Johnson    mia_johnson_49@mail.com  +91-7736163235\n   

                skills cgpa  \
0  python, sql, ml, ai        
1  python, sql, ml, ai        
2  python, sql, ml, ai        
3  python, sql, ml, ai        
4  python, sql, ml, ai        

                                                text               filename  
0  NAME: David Kim 91\nEmail: david_kim_91@mail.c...    ATS_DS_Resume_1.pdf  
1  NAME: Manisha Yadav 45\nEmail: manisha_yadav_4...   ATS_DS_Resume_10.pdf  
2  NAME: Ahmed Malik 47\nEmail: ahmed_malik_47@ma...  ATS_DS_Resume_100.pdf  
3  NAME: Mia Johnson 46\nEmail: mia_johnson_46@ma...   ATS_DS_Resume_11.pdf  
4  NAME: Mia Johnson 49\nEmail: 

In [16]:
import os
print("Current Working Dir:", os.getcwd())
print(os.listdir())


Current Working Dir: C:\Users\hp\Desktop\enhancement_NER_on100
['.ipynb_checkpoints', 'All_needed_entity.csv', 'app.py', 'ATS_DS_100_Unique_Names_Phones', 'cgpa.txt', 'emails.txt', 'names.txt', 'NER_100resumes.ipynb', 'NER_custom_model.ipynb', 'NER_custom_model.py', 'pdf_files', 'phones.txt', 'resumes_cleaned.csv', 'resumes_extracted.csv', 'resumes_extracted_new.csv', 'resume_ner_model_v1', 'skills.txt', 'sql_snowflake.ipynb', 'templates', 'train_candidates.json', 'ui.ipynb', 'untitled.py', 'upload_csv_to_s3.ipynb', 'upload_csv_to_s3.py', 'watcher.ipynb', '__pycache__']


In [17]:
df = pd.read_csv(r"resumes_extracted_new.csv")
df.head()


,filename,name,email,phone,skills,cgpa,text
0,ATS_DS_Resume_1.pdf,David Kim,david_kim_91@mail.com,+91-6393136116\n,"python, sql, ml, ai",NaN,NAME: David Kim 91\nEmail: david_kim_91@mail.c...
1,ATS_DS_Resume_10.pdf,Manisha Yadav,manisha_yadav_45@mail.com,+91-9610893319\n,"python, sql, ml, ai",NaN,NAME: Manisha Yadav 45\nEmail: manisha_yadav_4...
2,ATS_DS_Resume_100.pdf,Ahmed Malik,ahmed_malik_47@mail.com,+91-7718575010\n,"python, sql, ml, ai",NaN,NAME: Ahmed Malik 47\nEmail: ahmed_malik_47@ma...
3,ATS_DS_Resume_11.pdf,Mia Johnson,mia_johnson_46@mail.com,+91-8120603783\n,"python, sql, ml, ai",NaN,NAME: Mia Johnson 46\nEmail: mia_johnson_46@ma...
4,ATS_DS_Resume_12.pdf,Mia Johnson,mia_johnson_49@mail.com,+91-7736163235\n,"python, sql, ml, ai",NaN,NAME: Mia Johnson 49\nEmail: mia_johnson_49@ma...


In [18]:
df = pd.read_csv("resumes_extracted_new.csv")

# Remove filename & text columns
df = df.drop(['filename'], axis=1)

# Show final clean table
df


,name,email,phone,skills,cgpa,text
0,David Kim,david_kim_91@mail.com,+91-6393136116\n,"python, sql, ml, ai",NaN,NAME: David Kim 91\nEmail: david_kim_91@mail.c...
1,Manisha Yadav,manisha_yadav_45@mail.com,+91-9610893319\n,"python, sql, ml, ai",NaN,NAME: Manisha Yadav 45\nEmail: manisha_yadav_4...
2,Ahmed Malik,ahmed_malik_47@mail.com,+91-7718575010\n,"python, sql, ml, ai",NaN,NAME: Ahmed Malik 47\nEmail: ahmed_malik_47@ma...
3,Mia Johnson,mia_johnson_46@mail.com,+91-8120603783\n,"python, sql, ml, ai",NaN,NAME: Mia Johnson 46\nEmail: mia_johnson_46@ma...
4,Mia Johnson,mia_johnson_49@mail.com,+91-7736163235\n,"python, sql, ml, ai",NaN,NAME: Mia Johnson 49\nEmail: mia_johnson_49@ma...
...,...,...,...,...,...,...
104,Aman Verma SoftwareEngineer,aman.verma@example.com,+91-9876543210,"python, java, sql, flask",6.50,Aman Verma— SoftwareEngineer\n+91-9876543210|a...
105,NishthaGupta DataAnalyst,nishtha.gupta@example.com,+91-9123456789,"python, sql, excel, ai",7.42,NishthaGupta—DataAnalyst\n+91-9123456789|nisht...
106,RohitSingh FrontendDeveloper,rohit.singh@example.com,+91-9987654321,"java, ml, ai",8.63,RohitSingh—FrontendDeveloper\n+91-9987654321|r...
107,PriyaMenon MachineLearningEngineer,priya.menon@example.com,+91-9000011111,"python, sql, flask, ml, ai",8.95,PriyaMenon—MachineLearningEngineer\n+91-900001...


In [19]:
df.to_csv("All_needed_entity.csv", index=False)
print("Clean CSV saved as All_needed_entity.csv")


Clean CSV saved as All_needed_entity.csv


In [21]:
from upload_csv_to_s3 import upload_to_s3
upload_to_s3()


Uploaded to S3 → ner-output/All_needed_entity_1765279355.csv


In [20]:
# CSV created here
import csv
with open("All_needed_entity.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["name", "email", "phone", "skills", "cgpa"])
    writer.writerow([name, email, phone, skills, cgpa])

# Now upload to S3
upload_to_s3()


NameError: name 'name' is not defined

In [ ]:
import snowflake.connector

def upload_to_stage():
    conn = snowflake.connector.connect(...)
    cs = conn.cursor()
    
    cs.execute("""
        PUT file://All_needed_entity.csv @resume_stage
        OVERWRITE = TRUE
    """)
    
    cs.execute("""
        COPY INTO RESUME_DATA
        FROM @resume_stage
        FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1)
        ON_ERROR='CONTINUE'
    """)
    conn.commit()
    cs.close()
    conn.close()


In [25]:
import snowflake.connector

try:
    conn = snowflake.connector.connect(
        user="SejalChourey",
        password="Snowflakespass123",
        account="oxeywss-jk90862",
        warehouse="COMPUTE_WH",
        database="SALES_DB_NEW",
        schema="PUBLIC"
    )
    print("nowflake Connection Successful!")
except Exception as e:
    print("Connection Failed:", e)



✅ Snowflake Connection Successful!


In [22]:
import mysql.connector

# Step 1: Connect WITHOUT database
def connect_db_without_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

# Step 2: Create database IF NOT EXISTS
def create_database():
    db = connect_db_without_db()
    cursor = db.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS custom_ner_db")  
    db.commit()
    db.close()
    print("Database created or already exists.")

create_database()  # RUN THIS FIRST


✔ Database created or already exists.


In [24]:
import mysql.connector

def connect_db():
    return mysql.connector.connect(
        host="localhost",       
        user="root",            
        password="",            
        database="custom_ner_db"    
    )


In [26]:
db = connect_db()
cursor = db.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS resumes_info (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    email VARCHAR(255),
    phone VARCHAR(50),
    skills TEXT,
    cgpa VARCHAR(50)
);
"""
cursor.execute(create_table_query)
db.commit()
print("Table created successfully!")


✔ Table created successfully!


In [28]:
df = df.fillna("")   # NaN ko empty string me convert karega


In [30]:
df = df.where(pd.notnull(df), None)  # NaN → None (MySQL NULL)


In [32]:
df = df.fillna("")   # Step 1 - remove NaN

insert_query = """
INSERT INTO resumes_info (name, email, phone, skills, cgpa)
VALUES (%s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    values = (
        str(row['name']),
        str(row['email']),
        str(row['phone']),
        str(row['skills']),
        str(row['cgpa'])
    )
    cursor.execute(insert_query, values)

db.commit()
print("✔ All data inserted successfully!")


✔ All data inserted successfully!


In [34]:
print(df.columns)


Index(['name', 'email', 'phone', 'skills', 'cgpa', 'text'], dtype='object')


In [36]:
import re

def find_regex_spans(text):
    ents = []
    for m in re.finditer(r"\b[A-Z][a-z]+\s[A-Z][a-z]+\b", text):  # Two words names
        ents.append((m.start(), m.end(), "PERSON"))
    for m in re.finditer(r"\b[A-Z][a-z]{2,}\b", text):  # Single word names
        ents.append((m.start(), m.end(), "PERSON"))
    return ents

nlp_base = spacy.load("en_core_web_sm")

candidates = []
N_SEED = min(100, len(df))  
for i in range(N_SEED):
    text = str(df.loc[i, "text"])  # Make sure "text" column exists!
    ents = find_regex_spans(text)
    doc = nlp_base(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            ents.append((ent.start_char, ent.end_char, "PERSON"))

    unique = []
    seen = set()
    for s, e, l in sorted(ents, key=lambda x: (x[0], -x[1])):
        key = (s, e, l)
        if key not in seen:
            unique.append((s, e, l))
            seen.add(key)

    candidates.append((text, {"entities": unique}))

from pathlib import Path
import json

Path("train_candidates.json").write_text(
    json.dumps(candidates, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

print("Saved", len(candidates), "candidate examples to train_candidates.json")
print("✔ Open and correct offsets/labels for best results.")


NameError: name 'spacy' is not defined

In [38]:
train_candidates = json.loads(Path("train_candidates.json").read_text())

# shuffle and split (80/20)
random.shuffle(train_candidates)
split_idx = int(0.8 * len(train_candidates))
TRAIN_DATA = train_candidates[:split_idx]
DEV_DATA = train_candidates[split_idx:]

def filter_aligned(examples):
    kept, skipped = [], 0
    nlp_tmp = English()
    for text, ann in examples:
        try:
            biluo = offsets_to_biluo_tags(nlp_tmp.make_doc(text), ann["entities"])
            if "-" in biluo:
                skipped += 1
            else:
                kept.append((text, ann))
        except Exception as e:
            skipped += 1
    return kept, skipped

TRAIN_DATA, bad_train = filter_aligned(TRAIN_DATA)
DEV_DATA, bad_dev = filter_aligned(DEV_DATA)
print("Kept train:", len(TRAIN_DATA), "skipped:", bad_train)
print("Kept dev:", len(DEV_DATA), "skipped:", bad_dev)


NameError: name 'Path' is not defined

In [40]:
nlp = spacy.load("en_core_web_sm")
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# add labels
labels = set()
for _, ann in TRAIN_DATA:
    for s,e,l in ann["entities"]:
        labels.add(l)
for label in labels:
    ner.add_label(label)

other_pipes = [p for p in nlp.pipe_names if p != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.resume_training()
    for epoch in range(30):  
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], sgd=optimizer, drop=0.2, losses=losses)
        print(f"Epoch {epoch+1} Losses: {losses}")

# Save model
nlp.to_disk("resume_ner_model_v1")
print("Model saved to resume_ner_model_v1")


NameError: name 'spacy' is not defined

In [42]:
nlp_trained = spacy.load("resume_ner_model_v1")

for text, ann in DEV_DATA[:10]:
    print("---- SAMPLE ----")
    print(text[:300], "...\n")
    print("GOLD:", ann["entities"])
    pred = [(ent.start_char, ent.end_char, ent.label_, ent.text) for ent in nlp_trained(text).ents]
    print("PRED:", pred)
    print()


NameError: name 'spacy' is not defined

In [44]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [45]:
import spacy

import re

# Email regex
EMAIL_RE = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")

# Phone regex (Indian + International format)
PHONE_RE = re.compile(r"\b(?:\+?\d{1,3}[- ]?)?\d{10}\b")
CGPA_RE = re.compile(r"(?:CGPA[:\-]?\s*|CGPA\s*\(\s*|)(\d\.\d{1,2})")

COMMON_SKILLS = [
    "python", "java", "c++", "html", "css", "javascript", "sql", "machine learning",
    "deep learning", "django", "flask", "react", "tensorflow", "opencv",
    "power bi", "pandas", "numpy", "git", "communication", "leadership",
    "teamwork", "microsoft excel", "excel", "nlp", "data analysis"
]

SKILLS_RE = re.compile(r"\b(" + "|".join([re.escape(s) for s in COMMON_SKILLS]) + r")\b", re.IGNORECASE)
if not skills:
    skills = list({m.group(0).strip() for m in SKILLS_RE.finditer(text)})
# Name regex (First + Last Name with capital letters)
NAME_RE = re.compile(r"\b[A-Z][a-z]+\s[A-Z][a-z]+\b")

# Also support UPPERCASE names
UPPER_NAME_RE = re.compile(r"\b[A-Z]{2,}\s[A-Z]{2,}\b")

nlp_final = spacy.load("resume_ner_model_v1")  
nlp_default = spacy.load("en_core_web_sm")    

# reset output files
for fn in ["names.txt","phones.txt","emails.txt","skills.txt","cgpa.txt"]:
    Path(fn).write_text("")

def extract_name_spacy(text):
    """
    First try resume_ner_model_v1 → if PERSON not found,
    fallback to en_core_web_sm NER model
    """
    doc1 = nlp_final(text)  # YOUR TRAINED MODEL
    for ent in doc1.ents:
        if ent.label_ == "PERSON":
            return ent.text.strip()

    # fallback with default spacy model
    doc2 = nlp_default(text)
    for ent in doc2.ents:
        if ent.label_ == "PERSON":
            return ent.text.strip()

    return None  # nothing found

for idx, row in df.iterrows():
    text = str(row["text"])
    doc = nlp_final(text)

     # NAME extraction 
    name = extract_name_spacy(text)
    names = [name] if name else [] 

    # collect per doc
    #names = [ent.text.strip() for ent in doc.ents if ent.label_ == "PERSON"]
    emails = [ent.text.strip() for ent in doc.ents if ent.label_ == "EMAIL"]
    phones = [ent.text.strip() for ent in doc.ents if ent.label_ == "PHONE"]
    skills = [ent.text.strip() for ent in doc.ents if ent.label_ == "SKILL"]
    cgpas = [ent.text.strip() for ent in doc.ents if ent.label_ == "CGPA"]

    # fallbacks by regex if model missed
    if not emails:
        emails = [m.group(0) for m in EMAIL_RE.finditer(text)]
    if not phones:
        phones = [m.group(0) for m in PHONE_RE.finditer(text)]
    if not cgpas:
        cgpas = [m.group(1) for m in CGPA_RE.finditer(text)] 
    if not skills:
        skills = list({m.group(0) for m in SKILLS_RE.finditer(text)})

    # fallback
    if not names:
        name_match = re.search(NAME_RE, text)
        if name_match:
            names = [name_match.group()]

    # normalize and dedupe
    def clean_list(lst):
        return [s for s in dict.fromkeys([x.strip() for x in lst if x and x.strip()])]

    names = clean_list(names) or ["No Name Found"]
    emails = clean_list(emails) or ["No Email"]
    phones = clean_list(phones) or ["No Phone"]
    skills = clean_list(skills) or ["No Skills Found"]
    cgpas = clean_list(cgpas) or ["No CGPA Found"]

    # write per file lines with filename tag
    with open("names.txt","a",encoding="utf-8") as f:
        for v in names:
            f.write(f"{v}\n")

    with open("emails.txt","a",encoding="utf-8") as f:
        for v in emails:
            f.write(f"{v}\n")

    with open("phones.txt","a",encoding="utf-8") as f:
        for v in phones:
            f.write(f"{v}\n")

    with open("skills.txt","a",encoding="utf-8") as f:
        for v in skills:
            f.write(f"{v}\n")

    with open("cgpa.txt","a",encoding="utf-8") as f:
        for v in cgpas:
            f.write(f"{v}\n")

print("Done — check names.txt, phones.txt, emails.txt, skills.txt, cgpa.txt")


NameError: name 'skills' is not defined

In [ ]:
import mysql.connector

def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="Custom_NER_db"   # Create if not exists in next step
    )


In [ ]:
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)
cursor = db.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS Custom_NER_db")
cursor.execute("USE Custom_NER_db")

cursor.execute("""CREATE TABLE IF NOT EXISTS names_tbl (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255)
)""")

cursor.execute("""CREATE TABLE IF NOT EXISTS phones_tbl (
    id INT AUTO_INCREMENT PRIMARY KEY,
    phone VARCHAR(50)
)""")

cursor.execute("""CREATE TABLE IF NOT EXISTS emails_tbl (
    id INT AUTO_INCREMENT PRIMARY KEY,
    email VARCHAR(255)
)""")

cursor.execute("DROP TABLE IF EXISTS skills_tbl")

cursor.execute("""
CREATE TABLE skills_tbl (
    id INT AUTO_INCREMENT PRIMARY KEY,
    skills TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS cgpa_tbl (
    id INT AUTO_INCREMENT PRIMARY KEY,
    cgpa FLOAT
)
""")

db.commit()
print("Database and tables created!")


In [399]:
db = connect_db()
cursor = db.cursor()

with open("names.txt", "r") as f:
    for line in f:
        name = line.strip()
        cursor.execute("INSERT INTO names_tbl (name) VALUES (%s)", (name,))

db.commit()
print("Names inserted successfully!")


Names inserted successfully!


In [400]:
with open("phones.txt", "r") as f:
    for line in f:
        phone = line.strip()
        cursor.execute("INSERT INTO phones_tbl (phone) VALUES (%s)", (phone,))
db.commit()
print("Phones inserted successfully!")


Phones inserted successfully!


In [401]:
with open("emails.txt", "r") as f:
    for line in f:
        email = line.strip()
        cursor.execute("INSERT INTO emails_tbl (email) VALUES (%s)", (email,))
db.commit()
print("Emails inserted successfully!")


Emails inserted successfully!


In [402]:
with open("skills.txt", "r") as f:
    for line in f:
        skills = line.strip()
        cursor.execute("INSERT INTO skills_tbl (skills) VALUES (%s)", (skills,))
db.commit()
print("Skills inserted successfully!")


Skills inserted successfully!


In [403]:
with open("cgpa.txt", "r") as f:
    for line in f:
        cgpa = line.strip()
        cursor.execute("INSERT INTO cgpa_tbl (cgpa) VALUES (%s)", (cgpa,))

db.commit()
print("CGPA inserted successfully!")


CGPA inserted successfully!


In [404]:
def sanitize(value):
    if not value or str(value).strip() == "" or str(value).lower() == "none":
        return "NAN"
    return value

def insert_data(name, email, phone, skills, cgpa, address):
    db = connect_db()
    cursor = db.cursor()

    name = sanitize(name)
    email = sanitize(email)
    phone = sanitize(phone)
    skills = sanitize(skills)
    cgpa = sanitize(cgpa)
    address = sanitize(address)

    query = """
    INSERT INTO resumes_info (name, email, phone, skills, cgpa, address)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    values = (name, email, phone, skills, cgpa, address)
    
    cursor.execute(query, values)
    db.commit()
    cursor.close()
    db.close()
    print("✔ Record inserted successfully!")


In [487]:
!pip install mysql-connector-python
!pip install snowflake-connector-python


  Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl (3.5 MB)
  Attempting uninstall: cryptography
    Found existing installation: cryptography 42.0.5
    Uninstalling cryptography-42.0.5:
      Successfully uninstalled cryptography-42.0.5


In [488]:
!pip uninstall cryptography -y


Found existing installation: cryptography 46.0.3
Uninstalling cryptography-46.0.3:
  Successfully uninstalled cryptography-46.0.3


In [491]:
!pip install cryptography==42.0.5


  Using cached cryptography-42.0.5-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
Using cached cryptography-42.0.5-cp39-abi3-win_amd64.whl (2.9 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 25.3.0 requires cryptography<47,>=45.0.7, but you have cryptography 42.0.5 which is incompatible.
snowflake-connector-python 4.1.0 requires cryptography>=44.0.1, but you have cryptography 42.0.5 which is incompatible.


In [493]:
import mysql.connector
import snowflake.connector

# MySQL Connect
mysql_conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="Custom_NER_db"
)

mysql_cursor = mysql_conn.cursor(dictionary=True)
mysql_cursor.execute("SELECT * FROM resumes_info")
data = mysql_cursor.fetchall()


In [499]:
import snowflake.connector

sf_conn = snowflake.connector.connect(
    user='SejalChourey',3
    password='Snowflakespass123',
    account='oxeywss-jk90862',  
    warehouse='COMPUTE_WH',
    database='SALES_DB_NEW',
    schema='RAW'
)

sf_cursor = sf_conn.cursor()
print("Connected!")


Connected!


In [503]:
print(data[0])


{'id': 1, 'name': 'David Kim', 'email': 'david_kim_91@mail.com', 'phone': '+91-6393136116\n', 'skills': 'python, sql, ml, ai', 'cgpa': ''}


In [509]:
sf_cursor.execute("""
    CREATE OR REPLACE TABLE snowflake_table (
        id INT,
        name STRING,
        email STRING,
        phone STRING,
        skills STRING,
        cgpa STRING
    )
""")


In [511]:
for row in data:
    sf_cursor.execute("""
        INSERT INTO snowflake_table(id, name, email, phone, skills, cgpa)
        VALUES(%s, %s, %s, %s, %s, %s)
    """, (
        row['id'],
        row['name'],
        row['email'],
        row['phone'].strip(), 
        row['skills'],
        row['cgpa'] if row['cgpa'] else None  
    ))

sf_conn.commit()  
